# 「gihyo.jpの機械学習始めよう　第3回ベイジアンフィルタを実装してみよう」を実践
[giyo.jpのページ](http://gihyo.jp/dev/serial/01/machine-learning/0003?page=1)

In [1]:
# 必用なモジュールをimport
import sys
import math
# python3ではpip install mecab-python3
import MeCab

In [2]:
# MeCab解析器を分かち書きモードで取得
tagger = MeCab.Tagger("-Owakati")

def get_words(doc):
        """
        与えられた文字列docに対して単語分割を行い、タプルで返す。　
        doc = '単語分割を行う。'
        return ('単語', '分割', 'を', '行う',  '。')
        """
        tagged =  tagger.parse(doc)
        return tuple(word for word in tagged.rstrip().split(' '))

## ナイーブベイズクラス

文章(doc)からカテゴリ(cat)に属する確率をベイズの定理を用いて求める。

$$ P(cat | doc) = \frac{P(doc | cat) P(cat)}{P(doc)} $$

文章は分割した単語(word)の集合で、単語の独立性を仮定して以下のように近似する。

$$ P(doc | cat) = P(word1 | cat) P(word2 | cat) ... P(word\it{n}| cat) $$

In [3]:
class NaiveBayes:
    def __init__(self):
        self.vocabularies = set() # 単語の集合
        self.wordcount = {} # {カテゴリ1:{単語1:n, 単語2:n, ...}, ...}
        self.catcount = {} # {カテゴリ1:n, ...}
        
    def word_count_up(self, word, cat):        
        """あるカテゴリの単語頻度をカウントアップする"""
        self.wordcount.setdefault(cat, {}) # デフォルト値を設定
        self.wordcount[cat].setdefault(word, 0)
        self.wordcount[cat][word] += 1
        self.vocabularies.add(word)
        
    def cat_count_up(self, cat):
        """カテゴリの頻度をカウントアップする"""
        self.catcount.setdefault(cat, 0)
        self.catcount[cat] += 1
        
    def train(self, doc, cat):
        """モデルを訓練する"""
        words = get_words(doc)
        for word in words:
            self.word_count_up(word, cat)
        self.cat_count_up(cat)
        
    def prior_prob(self, cat):
        """catの事前確率P(cat)を求める"""
        return float(self.catcount[cat]) / sum(self.catcount.values())
    
    def incategory(self, word, cat):
        """あるカテゴリ(cat)中にある単語(word)が出現した回数を返す"""
        if word in self.wordcount[cat]:
            return float(self.wordcount[cat][word])
        return 0.0
    
    def wordprob(self, word, cat):
        """
        P(word | cat), 尤度を求める
        加算スムージング
        """
        prob = \
            (self.incategory(word, cat) + 1.0) / \
            (sum(self.wordcount[cat].values()) +  len(self.vocabularies) * 1.0) 
        return prob
    
    def score(self, words, cat):
        """
        確率値を対数で算出
        対数化することで足し算になる
        """
        score = math.log(self.prior_prob(cat))
        for word in words:
            score += math.log(self.wordprob(word, cat))
        return score
    
    def classifier(self, words):
        """
        分類
        もっとも高い確率のカテゴリを獲得
        """
        best = None
        max_prob =  -sys.maxsize
        
        for cat in self.catcount.keys():
            prob = self.score(words, cat)
            if prob > max_prob:
                max_prob = prob
                best = cat
        return best

# 訓練データを与えて分類を実行

In [4]:
nb = NaiveBayes()

In [5]:
# gihyo.jpの例を利用
nb.train('''Python（パイソン）は，オランダ人のグイド・ヴァンロッサムが作ったオープンソースのプログラミング言語。
オブジェクト指向スクリプト言語の一種であり，Perlとともに欧米で広く普及している。イギリスのテレビ局 BBC が製作したコメディ番組『空飛ぶモンティパイソン』にちなんで名付けられた。
Python は英語で爬虫類のニシキヘビの意味で，Python言語のマスコットやアイコンとして使われることがある。Pythonは汎用の高水準言語である。プログラマの生産性とコードの信頼性を重視して設計されており，核となるシンタックスおよびセマンティクスは必要最小限に抑えられている反面，利便性の高い大規模な標準ライブラリを備えている。
Unicode による文字列操作をサポートしており，日本語処理も標準で可能である。多くのプラットフォームをサポートしており（動作するプラットフォーム），また，豊富なドキュメント，豊富なライブラリがあることから，産業界でも利用が増えつつある。''', 'Python')

nb.train('''Ruby（ルビー）は，まつもとゆきひろ（通称Matz）により開発されたオブジェクト指向スクリプト言語であり，従来 Perlなどのスクリプト言語が用いられてきた領域でのオブジェクト指向プログラミングを実現する。Rubyは当初1993年2月24日に生まれ， 1995年12月にfj上で発表された。名称のRubyは，プログラミング言語Perlが6月の誕生石であるPearl（真珠）と同じ発音をすることから，まつもとの同僚の誕生石（7月）のルビーを取って名付けられた。''', 'Ruby')

nb.train('''豊富な機械学習（きかいがくしゅう，Machine learning）とは，人工知能における研究課題の一つで，人間が自然に行っている学習能力と同様の機能をコンピュータで実現させるための技術・手法のことである。ある程度の数のサンプルデータ集合を対象に解析を行い，そのデータから有用な規則，ルール，知識表現，判断基準などを抽出する。データ集合を解析するため，統計学との関連も非常に深い。
機械学習は検索エンジン，医療診断，スパムメールの検出，金融市場の予測，DNA配列の分類，音声認識や文字認識などのパターン認識，ゲーム戦略，ロボット，など幅広い分野で用いられている。応用分野の特性に応じて学習手法も適切に選択する必要があり，様々な手法が提案されている。それらの手法は， Machine Learning や IEEE Transactions on Pattern Analysis and Machine Intelligence などの学術雑誌などで発表されることが多い。''', '機械学習')

In [6]:
words = get_words('オランダ人が開発したオープンソースのプログラミング言語です。オブジェクト指向でPerlに似ている。')
pred_cat = nb.classifier(words)
print(pred_cat)

Python


In [7]:
words = get_words('かの有名なMatzが開発した日本初のスクリプト言語')
pred_cat = nb.classifier(words)
print(pred_cat)

Ruby


In [8]:
words = get_words('人工知能を用いた学習技術')
pred_cat = nb.classifier(words)
print(pred_cat)

機械学習


In [9]:
words = get_words('様々な機械学習ライブラリが存在するプログラミング言語である')
pred_cat = nb.classifier(words)
print(pred_cat)

Python


In [10]:
words = get_words('日本の人が開発したプログラミング言語')
pred_cat = nb.classifier(words)
print(pred_cat)

Python


In [11]:
words = get_words('日本人が開発したプログラミング言語')
pred_cat = nb.classifier(words)
print(pred_cat)

Ruby
